In [47]:
import requests
import csv
import json
import random
import datetime
import pandas as pd

In [3]:
def add_zeros(num):
    numberWithZeroes = str(num)
    counter = len(numberWithZeroes)

    while counter < 11:
        numberWithZeroes = "0" + numberWithZeroes
        counter += 1

    return numberWithZeroes  # Return the modified string with leading zeroes

In [48]:
def processa_csv(csv_file, json_file):

    data = {}
    data['listaIdentificadorConta'] = []
    counter = 0
    today = datetime.date.today()
    
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)
        
        for row in reader:
            cpf = {
                'codigoIdentificadorUnicoContaBancoCentralBrasil': add_zeros(row[0])
            }
            data['listaIdentificadorConta'].append(cpf)
            counter += 1
    
    data['quantidadeListaIdentificadorConta'] = counter  # Add count to JSON
    
    

    for cpf in data['listaIdentificadorConta']:
        cpf['pix_existe'] = 'X'
        cpf['data_validacao'] = datetime.datetime.now().strftime('%d/%m/%Y')
        if 'data_validacao' not in cpf or datetime.datetime.strptime(cpf['data_validacao'], '%d/%m/%Y').date() < today - datetime.timedelta(days=30):
            valida(cpf) # For each validate if has pix
    
    with open(json_file, 'w') as file: # Open the JSON file for writing
        json.dump(data, file, indent=4)  # Write the JSON data to the output file with indentation

In [46]:
def valida(cpf):
    url = 'https://pbn-controle-pagamento-beneficio.pbn.intranet.bb.com.br/validacao/validarChavePix'
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=cpf)
    content = response.json()

    return content

In [52]:
json_data = '''
{
    "listaIdentificadorConta": [
        {
            "codigoIdentificadorUnicoContaBancoCentralBrasil": "04357825172",
            "pix_existe": "X",
            "data_validacao": "30/05/2023"
        },
        {
            "codigoIdentificadorUnicoContaBancoCentralBrasil": "20138717672",
            "pix_existe": "X",
            "data_validacao": "30/05/2023"
        }
    ],
    "quantidadeListaIdentificadorConta": 2
}
'''

parsed_data = json.loads(json_data)
new_lista_identificador_conta = []

for item in parsed_data['listaIdentificadorConta']:
    new_item = {
        'codigoIdentificadorUnicoContaBancoCentralBrasil': item['codigoIdentificadorUnicoContaBancoCentralBrasil']
    }
    new_lista_identificador_conta.append(new_item)

new_json_data = {
    'listaIdentificadorConta': new_lista_identificador_conta,
    'quantidadeListaIdentificadorConta': parsed_data['quantidadeListaIdentificadorConta']
}

new_json_string = json.dumps(new_json_data, indent=4)
print(new_json_string)

{
    "listaIdentificadorConta": [
        {
            "codigoIdentificadorUnicoContaBancoCentralBrasil": "04357825172"
        },
        {
            "codigoIdentificadorUnicoContaBancoCentralBrasil": "20138717672"
        }
    ],
    "quantidadeListaIdentificadorConta": 2
}


In [49]:
csv_file = 'cpfs.csv'  # Replace with your CSV file name
json_file = 'cpfs.json'  # Replace with the desired output JSON file name

processa_csv(csv_file, json_file)